In [1]:
import boto3

In [2]:
email = "christopherkelly@engi.network"
from_addr = email

In [35]:
contact_list = "engi-newsletter"
template_name = "engi-newsletter-welcome-template"
test_topic = "engi-test"

In [4]:
ses_client = boto3.client("sesv2")

### Celery SQS backend

See https://docs.celeryq.dev/en/stable/getting-started/backends-and-brokers/sqs.html#broker-sqs

Be careful about having multiple Celery workers dequeueing jobs from the same queue!

In [5]:
sqs_client = boto3.client("sqs")

In [6]:
sqs_client.list_queues()

{'QueueUrls': ['https://us-west-2.queue.amazonaws.com/163803973373/celery',
  'https://us-west-2.queue.amazonaws.com/163803973373/dev-email-server-celery',
  'https://us-west-2.queue.amazonaws.com/163803973373/e969e804-d411-48ef-b050-df6afcced3da-same-story-test-queue',
  'https://us-west-2.queue.amazonaws.com/163803973373/prod-email-server-celery',
  'https://us-west-2.queue.amazonaws.com/163803973373/same-story-api',
  'https://us-west-2.queue.amazonaws.com/163803973373/same-story-check-queue'],
 'ResponseMetadata': {'RequestId': 'd646e254-f674-5b63-a3a4-b8a7bde16f0c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd646e254-f674-5b63-a3a4-b8a7bde16f0c',
   'date': 'Mon, 09 May 2022 17:03:17 GMT',
   'content-type': 'text/xml',
   'content-length': '825'},
  'RetryAttempts': 0}}

In [12]:
sqs_client.delete_queue(QueueUrl="https://us-west-2.queue.amazonaws.com/163803973373/celery-email-server-devcelery")

{'ResponseMetadata': {'RequestId': 'd2087a39-6373-51b8-a351-2086db9ae309',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd2087a39-6373-51b8-a351-2086db9ae309',
   'date': 'Thu, 31 Mar 2022 14:11:53 GMT',
   'content-type': 'text/xml',
   'content-length': '211'},
  'RetryAttempts': 0}}

In [5]:
topics = [
    {
        'TopicName': 'engi-programmer',
        'DisplayName': 'Engi Programmers',
        'Description': 'updates of specific interest for programmers',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-business',
        'DisplayName': 'Engi Business',
        'Description': 'updates of specific interest for business',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-investor',
        'DisplayName': 'Engi Investor',
        'Description': 'updates of specific interest for investors',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-curious',
        'DisplayName': 'Engi Curious',
        'Description': 'updates of interest for the curious',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-newsletter',
        'DisplayName': 'The Engi Newsletter',
        'Description': 'subscribe to updates about the development of engi',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
]

### Templates

In [5]:
client.list_email_templates()

{'ResponseMetadata': {'RequestId': '0d86d247-1f9e-40ab-a95f-71386d04c232',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 00:46:40 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0d86d247-1f9e-40ab-a95f-71386d04c232'},
  'RetryAttempts': 0},
 'TemplatesMetadata': [{'TemplateName': 'engi-newsletter-welcome-template',
   'CreatedTimestamp': datetime.datetime(2021, 10, 10, 12, 59, 20, tzinfo=tzlocal())}]}

In [6]:
client.get_email_template(TemplateName=template_name)

{'ResponseMetadata': {'RequestId': '24f5b140-82b3-4e4e-81bd-945a65964423',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 30 Mar 2022 23:51:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '235',
   'connection': 'keep-alive',
   'x-amzn-requestid': '24f5b140-82b3-4e4e-81bd-945a65964423'},
  'RetryAttempts': 0},
 'TemplateName': 'engi-newsletter-welcome-template',
 'TemplateContent': {'Subject': 'Welcome to Engi',
  'Text': 'Hi friend!\nThanks for subscribing to Engi updates.',
  'Html': '<h1>Hi friend!</h1><p>Thanks for subscribing to Engi updates.</p>'}}

In [7]:
client.update_email_template(
    TemplateName=template_name,
    TemplateContent={
        'Subject': 'Welcome to Engi',
        'Text': 'Hi friend!\nThanks for subscribing to Engi updates.',
        'Html': '<h1>Hi friend!</h1><p>Thanks for subscribing to Engi updates.</p>'
    }
)

{'ResponseMetadata': {'RequestId': '5a77b059-c6f6-4b64-a738-637fe57c375b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 30 Mar 2022 23:37:34 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5a77b059-c6f6-4b64-a738-637fe57c375b'},
  'RetryAttempts': 0}}

### Contact Lists

In [5]:
ses_client.list_contact_lists()

{'ResponseMetadata': {'RequestId': '65e9d43b-9c27-4dbd-b839-cc85619e2d72',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 28 May 2022 13:51:13 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'connection': 'keep-alive',
   'x-amzn-requestid': '65e9d43b-9c27-4dbd-b839-cc85619e2d72'},
  'RetryAttempts': 0},
 'ContactLists': [{'ContactListName': 'engi-newsletter',
   'LastUpdatedTimestamp': datetime.datetime(2022, 3, 11, 6, 13, 22, 579000, tzinfo=tzlocal())}]}

In [65]:
r = ses_client.get_contact_list(ContactListName=contact_list); r

{'ResponseMetadata': {'RequestId': '47a179fd-fc37-4e73-91e4-af13688b0200',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 14:06:54 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '1095',
   'connection': 'keep-alive',
   'x-amzn-requestid': '47a179fd-fc37-4e73-91e4-af13688b0200'},
  'RetryAttempts': 0},
 'ContactListName': 'engi-newsletter',
 'Topics': [{'TopicName': 'engi-programmer',
   'DisplayName': 'Engi Programmers',
   'Description': 'updates of specific interest for programmers',
   'DefaultSubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-business',
   'DisplayName': 'Engi Business',
   'Description': 'updates of specific interest for business',
   'DefaultSubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-investor',
   'DisplayName': 'Engi Investor',
   'Description': 'updates of specific interest for investors',
   'DefaultSubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-curious',
   'DisplayName': 'Engi Curious',

In [66]:
r['Topics']

[{'TopicName': 'engi-programmer',
  'DisplayName': 'Engi Programmers',
  'Description': 'updates of specific interest for programmers',
  'DefaultSubscriptionStatus': 'OPT_IN'},
 {'TopicName': 'engi-business',
  'DisplayName': 'Engi Business',
  'Description': 'updates of specific interest for business',
  'DefaultSubscriptionStatus': 'OPT_IN'},
 {'TopicName': 'engi-investor',
  'DisplayName': 'Engi Investor',
  'Description': 'updates of specific interest for investors',
  'DefaultSubscriptionStatus': 'OPT_IN'},
 {'TopicName': 'engi-curious',
  'DisplayName': 'Engi Curious',
  'Description': 'updates of interest for the curious',
  'DefaultSubscriptionStatus': 'OPT_IN'},
 {'TopicName': 'engi-newsletter',
  'DisplayName': 'The Engi Newsletter',
  'Description': 'subscribe to updates about the development of engi',
  'DefaultSubscriptionStatus': 'OPT_IN'},
 {'TopicName': 'engi-test',
  'DisplayName': 'Engi Test',
  'Description': 'testing',
  'DefaultSubscriptionStatus': 'OPT_OUT'}]

In [45]:
r = ses_client.update_contact_list(
    ContactListName=contact_list,
    Topics=[
        {
            "TopicName": "engi-programmer",
            "DisplayName": "Engi Programmers",
            "Description": "updates of specific interest for programmers",
            "DefaultSubscriptionStatus": "OPT_IN",
        },
        {
            "TopicName": "engi-business",
            "DisplayName": "Engi Business",
            "Description": "updates of specific interest for business",
            "DefaultSubscriptionStatus": "OPT_IN",
        },
        {
            "TopicName": "engi-investor",
            "DisplayName": "Engi Investor",
            "Description": "updates of specific interest for investors",
            "DefaultSubscriptionStatus": "OPT_IN",
        },
        {
            "TopicName": "engi-curious",
            "DisplayName": "Engi Curious",
            "Description": "updates of interest for the curious",
            "DefaultSubscriptionStatus": "OPT_IN",
        },
        {
            "TopicName": "engi-newsletter",
            "DisplayName": "The Engi Newsletter",
            "Description": "subscribe to updates about the development of engi",
            "DefaultSubscriptionStatus": "OPT_IN",
        },
        {
            "TopicName": test_topic,
            "DisplayName": "Engi Test",
            "Description": "testing",
            "DefaultSubscriptionStatus": "OPT_OUT",
        },
    ],
); r

{'ResponseMetadata': {'RequestId': '191a6fe5-9361-4358-82fc-65d679e45107',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:48:14 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '191a6fe5-9361-4358-82fc-65d679e45107'},
  'RetryAttempts': 0}}

In [10]:
r = ses_client.create_contact_list(
    ContactListName="testing",
    Topics=r['Topics'],
    Description="testing",
)

BadRequestException: An error occurred (BadRequestException) when calling the CreateContactList operation: A maximum of 1 Lists allowed per account.

In [49]:
ses_client.list_contacts(ContactListName=contact_list)

{'ResponseMetadata': {'RequestId': '31b89618-9207-4cca-9782-20cafc712492',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:53:59 GMT',
   'content-type': 'application/json',
   'content-length': '4172',
   'connection': 'close',
   'x-amzn-requestid': '31b89618-9207-4cca-9782-20cafc712492'},
  'RetryAttempts': 0},
 'Contacts': [{'EmailAddress': 'keith1015+engi@gmail.com',
   'TopicDefaultPreferences': [{'TopicName': 'engi-programmer',
     'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-investor', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-test', 'SubscriptionStatus': 'OPT_OUT'}],
   'UnsubscribeAll': False,
   'LastUpdatedTimestamp': datetime.datetime(2022, 5, 26, 23, 30, 46, 92000, tzinfo=tzlocal())},
  {'EmailAddress': 'ficegon536@nife

In [58]:
topic = contact_list
topic = test_topic
r = ses_client.list_contacts(
    ContactListName=contact_list,
    Filter={
        'FilteredStatus': 'OPT_IN',
        'TopicFilter': {
            'TopicName': topic,
            'UseDefaultIfPreferenceUnavailable': True
        }
    },
    PageSize=10,
); r

{'ResponseMetadata': {'RequestId': 'a7c831af-6866-4786-af09-e663254b6516',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:56:33 GMT',
   'content-type': 'application/json',
   'content-length': '551',
   'connection': 'close',
   'x-amzn-requestid': 'a7c831af-6866-4786-af09-e663254b6516'},
  'RetryAttempts': 0},
 'Contacts': [{'EmailAddress': 'christopherkelly@engi.network',
   'TopicPreferences': [{'TopicName': 'engi-test',
     'SubscriptionStatus': 'OPT_IN'}],
   'TopicDefaultPreferences': [{'TopicName': 'engi-programmer',
     'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-investor', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'}],
   'UnsubscribeAll': False,
   'LastUpdatedTimestamp': datetime.datetime(2022, 5, 31, 6, 55, 25, 97000, tzinfo=tzlocal())}]}

In [40]:
r

{'ResponseMetadata': {'RequestId': '563dd851-acad-4799-889e-5d6a10a00a5c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:41:12 GMT',
   'content-type': 'application/json',
   'content-length': '32',
   'connection': 'close',
   'x-amzn-requestid': '563dd851-acad-4799-889e-5d6a10a00a5c'},
  'RetryAttempts': 0},
 'Contacts': []}

In [14]:
r = client.list_contacts(ContactListName=contact_list, NextToken=r["NextToken"])

In [8]:
r = client.update_contact_list(
    ContactListName=contact_list,
    Topics=topics,
)

In [9]:
r

{'ResponseMetadata': {'RequestId': 'aac878c3-e398-487b-a1a4-36e81891dad2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 11 Mar 2022 14:13:22 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'aac878c3-e398-487b-a1a4-36e81891dad2'},
  'RetryAttempts': 0}}

### Contact Management

In [51]:
r = ses_client.create_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': 'a543606e-da63-43a7-a66c-a418f87ba605',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:55:03 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a543606e-da63-43a7-a66c-a418f87ba605'},
  'RetryAttempts': 0}}

In [64]:
r = ses_client.delete_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': '3be5433d-e905-45ec-ba76-40cb9009a02e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 14:01:04 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3be5433d-e905-45ec-ba76-40cb9009a02e'},
  'RetryAttempts': 0}}

In [52]:
r = ses_client.get_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': 'e50c6f9d-83c5-4b36-8a87-756394fbb09c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:55:11 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '615',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e50c6f9d-83c5-4b36-8a87-756394fbb09c'},
  'RetryAttempts': 0},
 'ContactListName': 'engi-newsletter',
 'EmailAddress': 'christopherkelly@engi.network',
 'TopicDefaultPreferences': [{'TopicName': 'engi-programmer',
   'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-investor', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-test', 'SubscriptionStatus': 'OPT_OUT'}],
 'UnsubscribeAll': False,
 'CreatedTimestamp': datetime.datetime(2022, 5, 31, 6, 55, 3, 310000, tzinfo=tzlocal()),
 'LastUpdatedTi

In [53]:
r = ses_client.update_contact(
    ContactListName=contact_list,
    EmailAddress=email,
    TopicPreferences=[
        {
            'TopicName': 'engi-test',
            'SubscriptionStatus': 'OPT_IN'
        },
    ],
    UnsubscribeAll=False,
    AttributesData='{"name": "chris", "favoriteanimal": "bonobo"}'
); r

{'ResponseMetadata': {'RequestId': 'b9804bad-c7d1-412b-97f9-f061f8b0527a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:55:25 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b9804bad-c7d1-412b-97f9-f061f8b0527a'},
  'RetryAttempts': 0}}

In [55]:
email

'christopherkelly@engi.network'

In [57]:
r = ses_client.get_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': '33726a53-67e2-444a-a799-0424c73d3b78',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:56:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '663',
   'connection': 'keep-alive',
   'x-amzn-requestid': '33726a53-67e2-444a-a799-0424c73d3b78'},
  'RetryAttempts': 0},
 'ContactListName': 'engi-newsletter',
 'EmailAddress': 'christopherkelly@engi.network',
 'TopicPreferences': [{'TopicName': 'engi-test',
   'SubscriptionStatus': 'OPT_IN'}],
 'TopicDefaultPreferences': [{'TopicName': 'engi-programmer',
   'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-investor', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'}],
 'UnsubscribeAll': False,
 'AttributesData': '{"name": "chris", "favoriteanimal": "bonobo"}',
 'CreatedTime

In [16]:
[d["TopicName"] for d in r["TopicPreferences"]]

['engi-newsletter']

In [18]:
r["ResponseMetadata"]["HTTPStatusCode"]

200

In [7]:
r = client.delete_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': 'bbe432cd-3a32-43c0-bcf5-4f62a2ac529e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 May 2022 19:32:20 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bbe432cd-3a32-43c0-bcf5-4f62a2ac529e'},
  'RetryAttempts': 0}}

### Sending Email

In [13]:
from_addr = "g@engi.network"
to_addr = "christopherkelly@engi.network"

#### SendEmail

In [14]:
r = ses_client.send_email(
    FromEmailAddress=from_addr,
    Destination={
        'ToAddresses': [
            to_addr,
        ],
    },
    Content={
        'Simple': {
            'Subject': {
                'Data': 'test 1',
                'Charset': 'UTF-8'
            },
            'Body': {
                'Text': {
                    'Data': 'body',
                    'Charset': 'UTF-8'
                },
                'Html': {
                    'Data': '<p>body</p>',
                    'Charset': 'UTF-8'
                }
            }
        },


    },
    ListManagementOptions={
        'ContactListName': contact_list,
        'TopicName': contact_list
    }
); r

{'ResponseMetadata': {'RequestId': 'ec35de3e-ab0f-4efa-b7a4-353577bf7359',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 May 2022 13:15:43 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ec35de3e-ab0f-4efa-b7a4-353577bf7359'},
  'RetryAttempts': 0},
 'MessageId': '010101811a42065b-fa1470d2-bddc-4dfa-9534-9f34514ef216-000000'}

In [7]:
r

{'ResponseMetadata': {'RequestId': '715c7a1a-8034-420e-a7dc-27786e806644',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 27 May 2022 19:23:19 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': '715c7a1a-8034-420e-a7dc-27786e806644'},
  'RetryAttempts': 0},
 'MessageId': '0101018106f91fbd-cafcdfc8-e4dd-4dbc-9166-6046cf110d0a-000000'}

In [59]:
import sys

In [60]:
sys.path.append('/Users/cck197/Documents/engi-network/email-server-api/email')

In [61]:
%run ../../email/bulk_email.py

In [63]:
list(list_contacts(contact_list, "engi-test"))

[[{'EmailAddress': 'christopherkelly@engi.network',
   'TopicPreferences': [{'TopicName': 'engi-test',
     'SubscriptionStatus': 'OPT_IN'}],
   'TopicDefaultPreferences': [{'TopicName': 'engi-programmer',
     'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-investor', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'}],
   'UnsubscribeAll': False,
   'LastUpdatedTimestamp': datetime.datetime(2022, 5, 31, 6, 55, 25, 97000, tzinfo=tzlocal()),
   'AttributesData': '{"name": "chris", "favoriteanimal": "bonobo"}'}]]

In [11]:
%run ../../email/send_welcome_email.py

In [12]:
email='christopherkelly@engi.network'; attrs='{"name": "chris", "favoriteanimal": "bonobo"}'

In [19]:
FROM_EMAIL = from_addr

In [20]:
FROM_EMAIL

'g@engi.network'

In [21]:
def send_welcome_email(to_addr, attributes):
    return ses_client.send_email(
        FromEmailAddress=FROM_EMAIL,
        Destination={
            "ToAddresses": [
                to_addr,
            ],
        },
        Content={
            "Template": {
                "TemplateName": TEMPLATE_NAME,
                "TemplateData": attributes,
            },
        },
        ListManagementOptions={
            "ContactListName": CONTACT_LIST_NAME,
            "TopicName": CONTACT_LIST_NAME,
        },
    )

In [22]:
send_welcome_email(email, attrs)

{'ResponseMetadata': {'RequestId': '21bc3ead-db39-4c39-a73a-3b0ab1521578',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 27 May 2022 19:29:31 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': '21bc3ead-db39-4c39-a73a-3b0ab1521578'},
  'RetryAttempts': 0},
 'MessageId': '0101018106fece30-cc60b955-8f1b-47fd-992f-207c42d04fa1-000000'}

In [ ]:
# use send_bulk_email

In [15]:
template_name

'engi-newsletter-welcome-template'

In [19]:
%pdb

Automatic pdb calling has been turned OFF


In [12]:
to_addr = email

In [16]:
contact_list

'engi-newsletter-test'

In [14]:
r = client.send_email(
        FromEmailAddress=from_addr,
        Destination={
            "ToAddresses": [
                to_addr,
            ],
        },
        Content={
            "Template": {
                "TemplateName": template_name,
                "TemplateData": "{}",
            },
        },
        ListManagementOptions={
            "ContactListName": contact_list,
            "TopicName": contact_list,
        }
)

NotFoundException: An error occurred (NotFoundException) when calling the SendEmail operation: List with name: engi-newsletter-test doesn't exist.

In [ ]:
r = client.send_email(
    FromEmailAddress=from_addr,
    Destination={
        'ToAddresses': [
            to_addr,
        ],
    },
    Content={
        'Template': {
            'TemplateName': template_name,
            'TemplateData': '{"name": "chris", "favoriteanimal": "bonobo"}'
        },
    },
    ListManagementOptions={
        'ContactListName': contact_list,
        'TopicName': contact_list
    }
)

In [ ]:
r

{'ResponseMetadata': {'RequestId': '60866212-76a5-4d86-9fce-7157f48f0e0b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 00:51:27 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': '60866212-76a5-4d86-9fce-7157f48f0e0b'},
  'RetryAttempts': 0},
 'MessageId': '0101017f714f27f4-74722b9e-a792-4ca1-bbeb-311a5e4c8bcd-000000'}

#### SendBulkEmail

In [41]:
contacts = list_contacts(client, contact_list, topic)

In [48]:
entries = [e for e in get_bulk_entries(contacts) if "kelly" in e["Destination"]["ToAddresses"][0]]

In [49]:
r = client.send_bulk_email(
    FromEmailAddress=from_addr,
    DefaultContent={
        'Template': {
            'TemplateName': template_name,
            'TemplateData': '{"name": "friend", "favoriteanimal": "not sure"}'
        }
    },
    BulkEmailEntries=entries,
); r

{'ResponseMetadata': {'RequestId': 'ed137a45-d649-4bf9-9eb0-4f4a751f5159',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 01:48:43 GMT',
   'content-type': 'application/json',
   'content-length': '136',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ed137a45-d649-4bf9-9eb0-4f4a751f5159'},
  'RetryAttempts': 0},
 'BulkEmailEntryResults': [{'Status': 'SUCCESS',
   'MessageId': '0101017f718394bf-628b0eb5-cdce-4685-a721-ca31e611151b-000000'}]}

In [2]:
results = [{'Status': 'SUCCESS',
   'MessageId': '0101017f718394bf-628b0eb5-cdce-4685-a721-ca31e611151b-000000'}]

In [7]:
all([k["Status"] == "SUCCESS" for k in results])

True

In [8]:
r

{'ResponseMetadata': {'RequestId': '9dba4950-a902-44bf-96c8-d93ff6e3b34c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 09 Mar 2022 19:09:26 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9dba4950-a902-44bf-96c8-d93ff6e3b34c'},
  'RetryAttempts': 0},
 'MessageId': '0101017f701607b8-0f106dd6-8da9-4405-89d7-bce6cb37cc50-000000'}

### Scratch Pad

In [9]:
dir(client.exceptions)

['AccountSuspendedException',
 'AlreadyExistsException',
 'BadRequestException',
 'ClientError',
 'ConcurrentModificationException',
 'ConflictException',
 'InvalidNextTokenException',
 'LimitExceededException',
 'MailFromDomainNotVerifiedException',
 'MessageRejected',
 'NotFoundException',
 'SendingPausedException',
 'TooManyRequestsException',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_code_to_exception',
 'from_code']

In [7]:
s = "{ \"name\":\"Alejandro\", \"favoriteanimal\": \"alligator\" }"

In [9]:
import json

In [11]:
json.loads(s)

{'name': 'Alejandro', 'favoriteanimal': 'alligator'}

In [14]:
json.dumps(json.dumps(_11))

'"{\\"name\\": \\"Alejandro\\", \\"favoriteanimal\\": \\"alligator\\"}"'

In [10]:
session = boto3.Session()

In [13]:
sqs_client = boto3.client("sqs")

In [15]:
sqs_client.list_queues()

{'ResponseMetadata': {'RequestId': 'e734cb36-9582-5571-84ea-9192a0d94383',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e734cb36-9582-5571-84ea-9192a0d94383',
   'date': 'Thu, 10 Mar 2022 17:19:23 GMT',
   'content-type': 'text/xml',
   'content-length': '228'},
  'RetryAttempts': 0}}

In [1]:
import pycurl

In [11]:
session.get_available_services()

['accessanalyzer',
 'account',
 'acm',
 'acm-pca',
 'alexaforbusiness',
 'amp',
 'amplify',
 'amplifybackend',
 'amplifyuibuilder',
 'apigateway',
 'apigatewaymanagementapi',
 'apigatewayv2',
 'appconfig',
 'appconfigdata',
 'appflow',
 'appintegrations',
 'application-autoscaling',
 'application-insights',
 'applicationcostprofiler',
 'appmesh',
 'apprunner',
 'appstream',
 'appsync',
 'athena',
 'auditmanager',
 'autoscaling',
 'autoscaling-plans',
 'backup',
 'backup-gateway',
 'batch',
 'braket',
 'budgets',
 'ce',
 'chime',
 'chime-sdk-identity',
 'chime-sdk-meetings',
 'chime-sdk-messaging',
 'cloud9',
 'cloudcontrol',
 'clouddirectory',
 'cloudformation',
 'cloudfront',
 'cloudhsm',
 'cloudhsmv2',
 'cloudsearch',
 'cloudsearchdomain',
 'cloudtrail',
 'cloudwatch',
 'codeartifact',
 'codebuild',
 'codecommit',
 'codedeploy',
 'codeguru-reviewer',
 'codeguruprofiler',
 'codepipeline',
 'codestar',
 'codestar-connections',
 'codestar-notifications',
 'cognito-identity',
 'cognito-i